In [ ]:
def submission_output(ids, prediction,name):
    df_ids = ids
    result = df_ids.to_frame()
    result["target"] = prediction
    result.to_csv('../predictions/'+name, index=False)
    return result

In [ ]:
def evaluate(model, train_features, train_labels, test_features, test_labels):
    predictions = model.predict(test_features)
    score = model.score(test_features,test_labels)
    print('Entrenamiento: {:0.4f}%'.format(model.score(train_features, train_labels)*100))
    print('Testeo: {:0.4f}%.'.format(score*100))
    print ("F1 Score: ", f1_score(test_labels, predictions))
    print ("F1 Score default: ", f1_score(test_labels, predictions))
   # print ("F1 Score: ", f1_score(test_labels, test_features["sentimiento"],average='micro'))

In [ ]:
def n_grams(list_, train,test, target):
    cosa = 0
    if (target == 1):
        cosa =1
    test["1-gram-"+str(cosa)]= 0
    test["2-gram-"+str(cosa)]= 0
    test["3-gram-"+str(cosa)]= 0
    train["1-gram-"+str(cosa)]= 0
    train["2-gram-"+str(cosa)]= 0
    train["3-gram-"+str(cosa)]= 0

    for x in list_:
        #print (x)
        fin =len(x)
        n_grams = 0
        while (n_grams < fin):
            grams = 0
            w =""
            
            if ((fin - n_grams) >0):
                if (grams != 0):
                    w+=" "
                w+=str(x[n_grams + grams])
                grams+=1
                test[str(grams)+"-gram-"+str(cosa)] = test[str(grams)+"-gram-"+str(cosa)] + test['text'].apply(lambda x: 1 if w in x else 0 )
                train[str(grams)+"-gram-"+str(cosa)] = train[str(grams)+"-gram-"+str(cosa)] + train['text'].apply(lambda x: 1 if w in x else 0 )
               # print (w)
                
            if ((fin - n_grams) > 1):
                w+=" "
                w+=str(x[n_grams + grams])
                grams+=1
                test[str(grams)+"-gram-"+str(cosa)] = test[str(grams)+"-gram-"+str(cosa)] + test['text'].apply(lambda x: 1 if w in x else 0 )
                train[str(grams)+"-gram-"+str(cosa)] = train[str(grams)+"-gram-"+str(cosa)] + train['text'].apply(lambda x: 1 if w in x else 0 )
               # print (w)
                
            if ((fin - n_grams) > 2):
                w+=" "
                w+=str(x[n_grams + grams])
                grams+=1
                test[str(grams)+"-gram-"+str(cosa)] = test[str(grams)+"-gram-"+str(cosa)] + test['text'].apply(lambda x: 1 if w in x else 0 )
                train[str(grams)+"-gram-"+str(cosa)] = train[str(grams)+"-gram-"+str(cosa)] + train['text'].apply(lambda x: 1 if w in x else 0 )
               # print (w)
               # print (" ")
            n_grams+=1

In [ ]:
def features(df_train, df_test):
    df_test['len']=df_test.text.map(lambda x: len(str(x)))#/280)
    df_test["word_count"] = df_test.text.map(lambda x: len(x.split(" ")))
    #df_test['importance']= df_test.text.map(lambda x: calculateTweetImportance(x))
    
    df_train['len']=df_train.text.map(lambda x: len(x))#/280)
    df_train["word_count"] = df_train.text.map(lambda x: len(x.split(" ")))
   
    #df_train['importance']= df_train.text.map(lambda x: calculateTweetImportance(x))

    # unique_word_count
    df_train['unique_word_count'] = df_train['text'].apply(lambda x: len(set(str(x).split())))
    df_test['unique_word_count'] = df_test['text'].apply(lambda x: len(set(str(x).split())))
    
    #df_train['unique_word_count'] = df_train['unique_word_count']/df_train["word_count"]
    #df_train["word_count"] = df_train["word_count"]/df_train["word_count"].max()
    
    #df_test['unique_word_count'] = df_test['unique_word_count']/df_test["word_count"]
    #df_test["word_count"] = df_test["word_count"]/df_test["word_count"].max()
    
    # url_count
    df_train['url_count'] = df_train['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
    df_test['url_count'] = df_test['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
    
    df_train["url_count"] = df_train["url_count"] > 0
    df_test["url_count"] = df_test["url_count"] > 0

    # mean_word_length
    df_train['mean_word_length'] = df_train['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
    df_test['mean_word_length'] = df_test['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

    # punctuation_count
    #df_train['punctuation_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
    #df_test['punctuation_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
    #df_train["punctuation_count"] = df_train["punctuation_count"] > 0
    #df_test["punctuation_count"] = df_test["punctuation_count"] > 0
    # hashtag_count
    df_train['hashtag_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c == '#']))
    df_test['hashtag_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c == '#']))

    #mention_count
    df_train['mention_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c == '@']))
    df_test['mention_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c == '@'])) 
    return None

In [ ]:
def binary(train, to_predict):
    key_ =list(train["keyword"].unique())
    train['Kindex']=train.keyword.map(lambda x:f'{( key_.index(x)):08b}')
    to_predict['Kindex']=to_predict.keyword.map(lambda x:f'{( key_.index(x)):08b}')

    bits=0
    Found=False
    while (not Found):
        x = pow(2, bits)
        if (len(key_) > x ):
            bits+=1
        else:
            Found= True
    bits

    for x in range(8):
        feat = 'key_bit'+str(x)
        train[feat] = 0
        to_predict[feat]=0

    for x in range(bits):
        train['key_bit'+str(x)]=train.Kindex.map(lambda y: int(y[x]))
        to_predict['key_bit'+str(x)]=to_predict.Kindex.map(lambda y: int(y[x]))

In [ ]:
def keyword_in_text_2(df):
    """
    RECIBE: un df
    DEVUELVE: el mismo df con una columna que indica si la keyword esta en el tweet.
    """
    
    df['contains_keyword'] = 0
    cant_filas = len(df.index)
    for y in range(cant_filas):
        df['contains_keyword'][y:]= str(df.iloc[y][0]).lower() in str(df.iloc[y][17]).lower()

In [ ]:
def keyword_in_text(df):
    """
    RECIBE: un df
    DEVUELVE: el mismo df con una columna que indica si la keyword esta en el tweet.
    """
    
    df['contains_keyword'] = 0
    cant_filas = len(df.index)
    for y in range(cant_filas):
        df['contains_keyword'][y:]= str(df.iloc[y][1]).lower() in str(df.iloc[y][3]).lower()

In [ ]:
def has_words(train, to_predict):  
    archivo = open("NW.txt", "r")
    for linea in archivo.readlines():
        train["Tiene_"+str(linea[:-1])] = train['text'].apply(lambda x: str(linea[:-1]) in str(x))
        to_predict["Tiene_"+str(linea[:-1])] = to_predict['text'].apply(lambda x: str(linea[:-1]) in str(x))
    archivo.close()

In [ ]:
def prob_words(train,to_predict,listaPalabras,dict_0,dict_1):
    to_predict["Puntaje_ser_1"]= 0
    to_predict["Puntaje_ser_0"]= 0
    train["Puntaje_ser_1"]= 0
    train["Puntaje_ser_0"]= 0
    cant_1=0
    cant_0=0
    for palabra in listaPalabras:
        if palabra in dict_0:
            cant_0 = dict_0[palabra]
        if palabra in dict_1:
            cant_1 = dict_1[palabra]
        prob_1 = cant_0 /(cant_1 +cant_0)
        prob_0 = cant_1 /(cant_1 +cant_0)
        to_predict["Prob_0_x"+str(palabra)] = to_predict.text.map(lambda x: prob_0 if (str(palabra) in str(x)) else 0 )
        to_predict["Puntaje_ser_0"]= to_predict["Puntaje_ser_0"] +  to_predict["Prob_0_x"+str(palabra)]\
        .map(lambda x: x )
        to_predict.drop(columns=["Prob_0_x"+str(palabra)], axis=1,inplace = True)
        
        to_predict["Prob_1_x"+str(palabra)] = to_predict.text.map(lambda x: prob_1 if (str(palabra) in str(x)) else 0 )
        to_predict["Puntaje_ser_1"]= to_predict["Puntaje_ser_1"] +  to_predict["Prob_1_x"+str(palabra)]\
        .map(lambda x: x )
        to_predict.drop(columns=["Prob_1_x"+str(palabra)], axis=1,inplace = True)
        
        train["Prob_0_x"+str(palabra)] = train.text.map(lambda x: prob_0 if (str(palabra) in str(x)) else 0 )
        train["Puntaje_ser_0"]= train["Puntaje_ser_0"] +  train["Prob_0_x"+str(palabra)]\
        .map(lambda x: x )
        train.drop(columns=["Prob_0_x"+str(palabra)], axis=1,inplace = True)
        
        train["Prob_1_x"+str(palabra)] = train.text.map(lambda x: prob_1 if (str(palabra) in str(x)) else 0 )
        train["Puntaje_ser_1"]= train["Puntaje_ser_1"] +  train["Prob_1_x"+str(palabra)]\
        .map(lambda x: x ) 
        train.drop(columns=["Prob_1_x"+str(palabra)], axis=1,inplace = True)
    

In [ ]:
# Removing punctuation, html tags, symbols, numbers, etc.
#has_words(train, to_predict)

def remove_noise(text):
    # Dealing with Punctuation
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

def remove(train, to_predict):
    #Calling remove_noise function in order to remove noise
    train['text'] = train['text'].apply(lambda x: remove_noise(x))
    train["text"]=(train.text.map(lambda x: (x.lower())))
    to_predict['text'] = to_predict['text'].apply(lambda x: remove_noise(x))
    to_predict["text"]=(to_predict.text.map(lambda x: (x.lower())))
    
    from nlppreprocess import NLP

    nlp = NLP()

    train['text'] = train['text'].apply(nlp.process)
    to_predict['text'] = to_predict['text'].apply(nlp.process)  

In [ ]:
def Has(df):
    #Posibles 1
    df["Tiene_key_impor"] = (df["keyword"] == "wreckage") | (df["keyword"] == "debris") | (df["keyword"] == "derailment") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Tiene_key_impor"].map(lambda x: 1  if (x == True) else 0)
    df["Tiene_key_impor"] = df["Tiene_key_impor"].map(lambda x: 1 if (x == True) else 0)

    df["Es_outbreak"] = (df["keyword"] == "outbreak") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_outbreak"].map(lambda x: 0.975  if (x == True) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_outbreak"].map(lambda x: 0.025  if (x == False) else 0)
    df["Es_outbreak"] = df["keyword"].map(lambda x: 0.975 if (x == True) else 0)
    df.drop(columns=["Es_outbreak"], axis=1,inplace = True)

    df["Es_oil_split"] = (df["keyword"] == "oil split") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_oil_split"].map(lambda x: 0.973  if (x == True) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_oil_split"].map(lambda x: 0.027  if (x == False) else 0)
    df["Es_oil_split"] = df["keyword"].map(lambda x: 0.973 if (x == True) else 0)
    df.drop(columns=["Es_oil_split"], axis=1,inplace = True)
    
    df["Es_typhoon"] = (df["keyword"] == "typhoon") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_typhoon"].map(lambda x: 0.973  if (x == True) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_typhoon"].map(lambda x: 0.027  if (x == False) else 0)
    df["Es_typhoon"] = df["keyword"].map(lambda x: 0.973 if (x == True) else 0)
    df.drop(columns=["Es_typhoon"], axis=1,inplace = True)

    df["Es_suicide_bombing"] = (df["keyword"] == "suicide bombing") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_suicide_bombing"].map(lambda x: 0.969  if (x == True) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_suicide_bombing"].map(lambda x: 0.031  if (x == False) else 0)
    df["Es_suicide_bombing"] = df["keyword"].map(lambda x: 0.969 if (x == True) else 0)
    df.drop(columns=["Es_suicide_bombing"], axis=1,inplace = True)

    df["Es_suicide_bomber"] = (df["keyword"] == "suicide bomber") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_suicide_bomber"].map(lambda x: 0.967  if (x == True) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_suicide_bomber"].map(lambda x: 0.033  if (x == False) else 0)
    df["Es_suicide_bomber"] = df["keyword"].map(lambda x: 0.967 if (x == True) else 0)
    df.drop(columns=["Es_suicide_bomber"], axis=1,inplace = True)

    df["Es_bombing"] = (df["keyword"] == "bombing") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_bombing"].map(lambda x: 0.931  if (x == True) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_bombing"].map(lambda x: 0.069  if (x == False) else 0)
    df["Es_bombing"] = df["keyword"].map(lambda x: 0.931 if (x == True) else 0)
    df.drop(columns=["Es_bombing"], axis=1,inplace = True)

    df["Es_suicide_bomb"] = (df["keyword"] == "suicide bomb") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_suicide_bomb"].map(lambda x: 0.914  if (x == True) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_suicide_bomb"].map(lambda x: 0.086  if (x == False) else 0)
    df["Es_suicide_bomb"] = df["keyword"].map(lambda x: 0.914 if (x == True) else 0)
    df.drop(columns=["Es_suicide_bomb"], axis=1,inplace = True)

    df["Es_rescuers"] = (df["keyword"] == "rescuers") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_rescuers"].map(lambda x: 0.914  if (x == True) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_rescuers"].map(lambda x: 0.086  if (x == False) else 0)
    df["Es_rescuers"] = df["keyword"].map(lambda x: 0.914 if (x == True) else 0)
    df.drop(columns=["Es_rescuers"], axis=1,inplace = True)

    df["Es_nuclear_disaster"] = (df["keyword"] == "nuclear disaster") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_nuclear_disaster"].map(lambda x: 0.911  if (x == True) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_nuclear_disaster"].map(lambda x: 0.089  if (x == False) else 0)
    df["Es_nuclear_disaster"] = df["keyword"].map(lambda x: 0.911 if (x == True) else 0)
    df.drop(columns=["Es_nuclear_disaster"], axis=1,inplace = True)


    #Posibles 0
    df["Tiene_key_no_impor"] = (df["keyword"] == "wreckage") | (df["keyword"] == "debris") | (df["keyword"] == "derailment") 
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Tiene_key_no_impor"].map(lambda x: 1  if (x == True) else 0)
    df["Tiene_key_no_impor"] =df["keyword"].map(lambda x: 1 if (x == True) else 0)

    df["Es_body_bags"] = (df["keyword"] == "body bags") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_body_bags"].map(lambda x: 0.025  if (x == False) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_body_bags"].map(lambda x: 0.975  if (x == True) else 0)
    df["Es_body_bags"] = df["keyword"].map(lambda x: 0.975 if (x == True) else 0)
    df.drop(columns=["Es_body_bags"], axis=1,inplace = True)

    df["Es_ruin"] = (df["keyword"] == "ruin") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_ruin"].map(lambda x: 0.027  if (x == False) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_ruin"].map(lambda x: 0.973  if (x == True) else 0)
    df["Es_ruin"] = df["keyword"].map(lambda x: 0.973 if (x == True) else 0)
    df.drop(columns=["Es_ruin"], axis=1,inplace = True)

    df["Es_blazing"] = (df["keyword"] == "blazing") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_blazing"].map(lambda x: 0.030  if (x == False) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_blazing"].map(lambda x: 0.970  if (x == True) else 0)
    df["Es_blazing"] = df["keyword"].map(lambda x: 0.970 if (x == True) else 0)
    df.drop(columns=["Es_blazing"], axis=1,inplace = True)

    df["Es_electrocute"] = (df["keyword"] == "electrocute") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_electrocute"].map(lambda x: 0.032  if (x == False) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_electrocute"].map(lambda x: 0.968  if (x == True) else 0)
    df["Es_electrocute"] = df["keyword"].map(lambda x: 0.968 if (x == True) else 0)
    df.drop(columns=["Es_electrocute"], axis=1,inplace = True)

    df["Es_traumatised"] = (df["keyword"] == "traumatised") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_traumatised"].map(lambda x: 0.058  if (x == False) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_traumatised"].map(lambda x: 0.942  if (x == True) else 0)
    df["Es_traumatised"] = df["keyword"].map(lambda x: 0.942 if (x == True) else 0)
    df.drop(columns=["Es_traumatised"], axis=1,inplace = True)

    df["Es_blew_up"] = (df["keyword"] == "blew up") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_blew_up"].map(lambda x: 0.061  if (x == False) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_blew_up"].map(lambda x: 0.939  if (x == True) else 0)
    df["Es_blew_up"] = df["keyword"].map(lambda x: 0.939 if (x == True) else 0)
    df.drop(columns=["Es_blew_up"], axis=1,inplace = True)

    df["Es_panicking"] = (df["keyword"] == "panicking") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_panicking"].map(lambda x: 0.061  if (x == False) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_panicking"].map(lambda x: 0.939  if (x == True) else 0)
    df["Es_panicking"] = df["keyword"].map(lambda x: 0.939 if (x == True) else 0)
    df.drop(columns=["Es_panicking"], axis=1,inplace = True)

    df["Es_body_bag"] = (df["keyword"] == "body bag") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_body_bag"].map(lambda x: 0.030  if (x == False) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_body_bag"].map(lambda x: 0.970  if (x == True) else 0)
    df["Es_body_bag"] = df["keyword"].map(lambda x: 0.970 if (x == True) else 0)
    df.drop(columns=["Es_body_bag"], axis=1,inplace = True)

    df["Es_screaming"] = (df["keyword"] == "screaming") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_screaming"].map(lambda x: 0.055  if (x == False) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_screaming"].map(lambda x: 0.945  if (x == True) else 0)
    df["Es_screaming"] = df["keyword"].map(lambda x: 0.945 if (x == True) else 0)
    df.drop(columns=["Es_screaming"], axis=1,inplace = True)

    df["Es_blight"] = (df["keyword"] == "blight") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_blight"].map(lambda x: 0.063  if (x == False) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_blight"].map(lambda x: 0.937  if (x == True) else 0)
    df["Es_blight"] = df["keyword"].map(lambda x: 0.937 if (x == True) else 0)
    df.drop(columns=["Es_blight"], axis=1,inplace = True)

    df["Es_wrecked"] = (df["keyword"] == "wrecked") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_wrecked"].map(lambda x: 0.077  if (x == False) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_wrecked"].map(lambda x: 0.923  if (x == True) else 0)
    df["Es_wrecked"] = df["keyword"].map(lambda x: 0.923 if (x == True) else 0)
    df.drop(columns=["Es_wrecked"], axis=1,inplace = True)

    df["Es_explode"] = (df["keyword"] == "explode") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_explode"].map(lambda x: 0.089  if (x == False) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_explode"].map(lambda x: 0.921  if (x == False) else 0)
    df["Es_explode"] = df["keyword"].map(lambda x: 0.921 if (x == True) else 0)
    df.drop(columns=["Es_explode"], axis=1,inplace = True)

    df["Es_epicentre"] = (df["keyword"] == "epicentre") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_epicentre"].map(lambda x: 0.083  if (x == False) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_epicentre"].map(lambda x: 0.917  if (x == True) else 0)
    df["Es_epicentre"] = df["keyword"].map(lambda x: 0.917 if (x == True) else 0)
    df.drop(columns=["Es_epicentre"], axis=1,inplace = True)

    df["Es_panic"] = (df["keyword"] == "panic") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_panic"].map(lambda x: 0.081  if (x == False) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_panic"].map(lambda x: 0.919  if (x == True) else 0)
    df["Es_panic"] = df["keyword"].map(lambda x: 0.919 if (x == True) else 0)
    df.drop(columns=["Es_panic"], axis=1,inplace = True)

    df["Es_bloody"] = (df["keyword"] == "bloody") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_bloody"].map(lambda x: 0.086  if (x == False) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_bloody"].map(lambda x: 0.914  if (x == True) else 0)
    df["Es_bloody"] = df["keyword"].map(lambda x: 0.914 if (x == True) else 0)
    df.drop(columns=["Es_bloody"], axis=1,inplace = True)

    df["Es_drown"] = (df["keyword"] == "drown") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_drown"].map(lambda x: 0.094  if (x == False) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_drown"].map(lambda x: 0.906  if (x == True) else 0)
    df["Es_drown"] = df["keyword"].map(lambda x: 0.906 if (x == True) else 0)
    df.drop(columns=["Es_drown"], axis=1,inplace = True)

    df["Es_stretcher"] = (df["keyword"] == "stretcher") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_stretcher"].map(lambda x: 0.091  if (x == False) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_stretcher"].map(lambda x: 0.909  if (x == True) else 0)
    df["Es_stretcher"] = df["keyword"].map(lambda x: 0.909 if (x == True) else 0)
    df.drop(columns=["Es_stretcher"], axis=1,inplace = True)

    df["Es_collide"] = (df["keyword"] == "collide") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_collide"].map(lambda x: 0.089  if (x == False) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_collide"].map(lambda x: 0.911  if (x == True) else 0)
    df["Es_collide"] = df["keyword"].map(lambda x: 0.911 if (x == True) else 0)
    df.drop(columns=["Es_collide"], axis=1,inplace = True)

    df["Es_smoke"] = (df["keyword"] == "smoke") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_smoke"].map(lambda x: 0.089  if (x == False) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_smoke"].map(lambda x: 0.911  if (x == True) else 0)
    df["Es_smoke"] = df["keyword"].map(lambda x: 0.911 if (x == True) else 0)
    df.drop(columns=["Es_smoke"], axis=1,inplace = True)

    df["Es_harm"] = (df["keyword"] == "harm") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_harm"].map(lambda x: 0.097  if (x == False) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_harm"].map(lambda x: 0.902  if (x == True) else 0)
    df["Es_harm"] = df["keyword"].map(lambda x: 0.903 if (x == True) else 0)
    df.drop(columns=["Es_harm"], axis=1,inplace = True)

    df["Es_drown"] = (df["keyword"] == "drown") 
    df["Puntaje_ser_1"]= df["Puntaje_ser_1"] + df["Es_drown"].map(lambda x: 0.094  if (x == False) else 0)
    df["Puntaje_ser_0"]= df["Puntaje_ser_0"] + df["Es_drown"].map(lambda x: 0.906  if (x == True) else 0)
    df["Es_drown"] = df["keyword"].map(lambda x: 0.906 if (x == True) else 0)
    df.drop(columns=["Es_drown"], axis=1,inplace = True)


In [ ]:
def asd(train):

    unos = train[train["target"]==1]
    ceros = train[train["target"]==0]
    y = []
    n = []
    unos["sepa"]=(unos.text.map(lambda x: (x.split())))
    ceros["sepa"] = (ceros.text.map(lambda x: (x.split())))
    for x in unos['sepa']:
        y.append(x)


    for i in ceros['sepa']:
        n.append(i)
    return y, n

### KNN

In [ ]:
def featuresKNN(df_train, df_test):
    df_test['len']=df_test.text.map(lambda x: len(x)/280)
    df_test["word_count"] = df_test.text.map(lambda x: len(x.split(" ")))
    df_test["word_count"] = df_test["word_count"]/df_test["word_count"].max()
    #df_test['importance']= df_test.text.map(lambda x: calculateTweetImportance(x))
    
    df_train['len']=df_train.text.map(lambda x: len(x)/280)
    df_train["word_count"] = df_train.text.map(lambda x: len(x.split(" ")))
    df_train["word_count"] = df_train["word_count"]/df_train["word_count"].max()
    #df_train['importance']= df_train.text.map(lambda x: calculateTweetImportance(x))

In [ ]:
def knn_best_distance(df,dist,x_train,y_train,x_test,y_test):
    for x in dist:
        print(x+" --------------------------")
        result , k = knn_predictor(x,x_train,y_train,x_test,y_test)
        
        df[x] = result

In [ ]:
def knn_predictor(distancia,x_train,y_train,x_test,y_test):
    kesimos = []
    result = []
    for k in range(2,40):
        print("con k: ", k)
        kesimos.append(k)
        knn = KNeighborsClassifier(n_neighbors = k, metric=distancia)
        knn.fit(x_train, y_train)
        res = predecir(knn,x_train,y_train,x_test,y_test)
        result.append(res)
    return result, kesimos

In [1]:
def predecir(model, train_features, train_labels, test_features, test_labels):
    score = model.score(test_features,test_labels)
    predict = model.predict(test_features)
    
    print('Entrenamiento: {:0.4f}%'.format(model.score(train_features, train_labels)*100))
    print('Testeo: {:0.4f}%.'.format(score*100))
    
    error1 = f1_score(test_labels, predict)
    print('F1 score: {:0.4f}.'.format(error1))
    print ("   ")
    return error1


### Comprobar

#### Predicciones Reales

In [ ]:
def comprobar(best_predict):
    test_labels = pd.read_csv("../Data/perfect_submission.csv")
    test_labels = test_labels["target"]
    print ("F1 Score: ", f1_score(test_labels, best_predict,average='micro'))
    print ("F1 Score default: ", f1_score(test_labels, best_predict))